<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">


# Resampling 

Resampling data is a very common operation when building a Modflow model. Usually it is used to project data from one grid onto the other. There are many different ways to do this. This notebook shows some examples of resampling methods that are incorporated in the `nlmod` package. These methods rely heavily on resampling methods in packages such as `rioxarray` and `scipy.interpolate`.

### Contents<a name="TOC"></a>
1. [Grid types](#gridtypes)
2. [Structured grid to fine structured grid](#2)
3. [Structured grid to locally refined grid](#3)
4. [Locally refined grid to structured grid](#4)
5. [Fill nan values](#5)
6. [Vector to grid](#6)
7. [Grid to reclist (stress period data)](#7)
8. [Real world example](#8)

In [ ]:
import nlmod
from nlmod.mdims import resample
import numpy as np
import xarray as xr
import flopy
import warnings


from matplotlib.colors import Normalize
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.geometry import Polygon as shp_polygon
from shapely.errors import ShapelyDeprecationWarning
from scipy.interpolate import RectBivariateSpline

from IPython.display import display

In [ ]:
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

print(f'nlmod version: {nlmod.__version__}')

nlmod.util.get_color_logger('INFO');

## [1. Grid types](#TOC)<a name="gridtypes"></a>

So far two different gridtypes are supported in `nlmod`:
- structured grids where the cellsize is fixed for all cells
- vertex grids where the cellsize differs locally. These grids are usually created using local grid refinement algorithms.

In this notebook we define a few xarray dataarray of structured and vertex grids. We use these grids in the next chapter to show the resampling functions in `nlmod`.

#### structured grid

This structured grid has random numbers between 0 and 9. Has 10 x 10 cells.

In [ ]:
# structured grid 2d
x = np.arange(1000, 1300, 100)
y = np.arange(20300, 20000, -100)
data_2d = np.random.randint(0, 10, size=(len(y), len(x)))
struc2d = xr.DataArray(data_2d, dims=('y', 'x'),
                       coords={'x': x,
                               'y': y})
fig, ax = plt.subplots()
ax.set_aspect('equal')
qm = struc2d.plot(ax=ax, lw=0.1, edgecolor='k')

##### structured grid with nan value

In [ ]:
struc2d_nan = struc2d.copy().astype(float)
struc2d_nan.values[0][1] = np.nan
fig, ax = plt.subplots()
ax.set_aspect('equal')
qm = struc2d_nan.plot(ax=ax, lw=0.1, edgecolor='k')

#### vertex grid

In [ ]:
dx = 100
dy = 100
x = np.arange(1000, 1300, dx)
y = np.arange(20300, 20000, -dy)
split_cell_no = 5

# create structured grid
xv, yv = np.meshgrid(x, y)
xvc = xv.ravel()
yvc = yv.ravel()
#xyi = np.stack((np.ravel(xv), np.ravel(yv)), axis=-1)

# create vertices
vertices = np.ones((len(xvc), 4, 2))
for i, x, y in zip(range(len(vertices)),xvc, yvc):
    vertices[i] = np.array([[x-(dx/2), y+(dy/2)],
                            [x+(dx/2), y+(dy/2)],
                            [x+(dx/2), y-(dy/2)],
                            [x-(dx/2), y-(dy/2)]])
    
# remove refined cell from structured grid
split_cell_x, split_cell_y = xvc[split_cell_no], yvc[split_cell_no]
xvc = np.delete(xvc, split_cell_no, 0)
yvc = np.delete(yvc, split_cell_no, 0)
vertices = np.delete(vertices, split_cell_no, 0)

# get cell centers of refined cell
x_refined = np.array([split_cell_x-(dx/4), split_cell_x+(dx/4), split_cell_x-(dx/4), split_cell_x+(dx/4)])
y_refined = np.array([split_cell_y+(dy/4), split_cell_y+(dy/4), split_cell_y-(dy/4), split_cell_y-(dy/4)])

# get vertices of refined cell
vert_refined = np.ones((len(x_refined), 4, 2))
for i, x, y in zip(range(len(vert_refined)),x_refined, y_refined):
    vert_refined[i] = np.array([[x-(dx/4), y+(dy/4)],
                                [x+(dx/4), y+(dy/4)],
                                [x+(dx/4), y-(dy/4)],
                                [x-(dx/4), y-(dy/4)]])

# add refined cell to the grid and vertices
xvc = np.insert(xvc, split_cell_no, x_refined, axis=0)
yvc = np.insert(yvc, split_cell_no, y_refined, axis=0)
vertices = np.insert(vertices, split_cell_no, vert_refined, axis=0)

# calculate_area
area_vertex = [(v[:,0].max() - v[:,0].min()) * (v[:,1].max() - v[:,1].min()) for v in vertices]

# get cellid
icell2d = np.arange(len(xvc))

# create values
values = np.random.randint(0, 10, size=len(icell2d))

# create vertextured dataarray
coords = dict(x=xr.DataArray(xvc, dims=['icell2d',]), y=xr.DataArray(yvc, dims=['icell2d',]))
vertex1 = xr.DataArray(values, dims=('icell2d'), coords=coords)
nlmod.plot.plot_vertex_array(vertex1, vertices, gridkwargs={'edgecolor': 'k'});

#### vertex grid with nan

In [ ]:
vertex1_nan = vertex1.copy().astype(float)
vertex1_nan.values[7] = np.nan

nlmod.plot.plot_vertex_array(vertex1_nan, vertices, gridkwargs={'edgecolor': 'k'});

## [2 Structured grid to fine structured grid](#TOC)<a name="2"></a>

In [ ]:
# new grid dimensions
dx = 50
xmid = np.arange(950+0.5*dx, 1250, 50)
ymid = np.arange(20350-0.5*dx, 20050, -dx)

In [ ]:
def compare_structured_data_arrays(da1, da2, method, edgecolor='k'):
    fig, axes = plt.subplots(ncols=2, figsize=(12,6))
    da1.plot(ax=axes[0], edgecolor=edgecolor, vmin=0, vmax=9)
    axes[0].set_aspect('equal')
    axes[0].set_title('original grid')
    da2.plot(ax=axes[1], edgecolor=edgecolor, vmin=0, vmax=9)
    axes[1].set_aspect('equal')
    axes[1].set_title(f'resampled grid, method {method}')

In [ ]:
ds = xr.Dataset(coords=dict(x=xmid, y=ymid), attrs=dict(gridtype='structured'))

### Without NaNs

In [ ]:
for method in ['nearest', 'linear', 'cubic', 'average', 'min']:
    if method == 'nearest':
        struc_out = resample.structured_da_to_ds(struc2d, ds, method=method)
    else:
        struc_out = resample.structured_da_to_ds(struc2d.astype(float), ds, method=method)        
    compare_structured_data_arrays(struc2d, struc_out, method)

### With NaNs

In [ ]:
for method in ['nearest', 'linear', 'cubic', 'average', 'mode']:
    struc_out = resample.structured_da_to_ds(struc2d_nan, ds, method=method)
    compare_structured_data_arrays(struc2d_nan, struc_out, method)

### Rectangular Bivariate Spline (not yet included in nlmod)

In [ ]:
interp_spline = RectBivariateSpline(struc2d.x.values, struc2d.y.values[::-1], struc2d.values[::-1], 
                                    ky=min(3,len(struc2d.y)-1), 
                                    kx=min(3,len(struc2d.x)-1))
arr_out = interp_spline(xmid, ymid[::-1], grid=True)[::-1]
struc_out = xr.DataArray(arr_out, dims=('y', 'x'),
                         coords={'x': xmid,
                                 'y': ymid})
compare_structured_data_arrays(struc2d, struc_out, 'Rectangular Bivariate Spline')

### Rectangular Bivariate Spline with nans (not yet included in nlmod)

In [ ]:
interp_spline = RectBivariateSpline(struc2d_nan.x.values, struc2d_nan.y.values[::-1], struc2d_nan.values[::-1], 
                                    ky=min(3,len(struc2d_nan.y)-1), 
                                    kx=min(3,len(struc2d_nan.x)-1))
interp_spline = RectBivariateSpline(struc2d_nan.x.values, struc2d_nan.y.values[::-1], struc2d_nan.values[::-1], 
                                    ky=min(3,len(struc2d_nan.y)-1), 
                                    kx=min(3,len(struc2d_nan.x)-1))
arr_out = interp_spline(xmid, ymid[::-1], grid=True)[::-1]
struc_out = xr.DataArray(arr_out, dims=('y', 'x'),
                         coords={'x': xmid,
                                 'y': ymid})
compare_structured_data_arrays(struc2d_nan, struc_out, 'Rectangular Bivariate Spline')

## [3. Structured grid to locally refined grid](#TOC)<a name="3"></a>

In [ ]:
def compare_struct_to_vertex(struc2d, res_vertex2d_n, vertices, method):
    fig, axes = plt.subplots(ncols=2, figsize=(12,6))
    struc2d.plot(ax=axes[0], edgecolor='k', vmin=0, vmax=9)
    axes[0].set_aspect('equal')
    axes[0].set_title('structured grid')
    nlmod.plot.plot_vertex_array(res_vertex2d_n, vertices, ax=axes[1], gridkwargs={'edgecolor': 'k'}, vmin=0, vmax=9)
    axes[1].set_title(f'locally refined grid, method {method}')

In [ ]:
data_vars = dict(area=(['icell2d'], area_vertex))
coords = dict(x=xr.DataArray(xvc, dims=['icell2d',]), y=xr.DataArray(yvc, dims=['icell2d',]))
attrs = dict(gridtype='vertex', extent=[950, 1250, 20050, 20350])
dsv = xr.Dataset(data_vars=data_vars, coords=coords, attrs=attrs)

### WIthout NaNs

In [ ]:
for method in ['nearest', 'linear', 'cubic']:
    res_vertex2d_n = resample.structured_da_to_ds(struc2d, dsv, method=method)
    compare_struct_to_vertex(struc2d, res_vertex2d_n, vertices, method)

## [4. Locally refined grid to structured grid](#TOC)<a name="4"></a>

In [ ]:
def compare_vertex_to_struct(vertex1, struc_out_n, method):
    fig, axes = plt.subplots(ncols=2, figsize=(12,6))
    nlmod.plot.plot_vertex_array(vertex1, vertices, ax=axes[0], gridkwargs={'edgecolor': 'k'}, vmin=0, vmax=9)
    axes[0].set_title('original')
    struc_out_n.plot(ax=axes[1], edgecolor='k', vmin=0, vmax=9)
    axes[1].set_title(f'resampled, method {method}')
    axes[1].set_aspect('equal')

### Without NaNs

In [ ]:
for method in ['nearest', 'linear', 'cubic']:
    struc_out_n = resample.vertex_da_to_ds(vertex1, ds=ds, method=method)
    compare_vertex_to_struct(vertex1, struc_out_n, method)

### With NaNs

In [ ]:
for method in ['nearest', 'linear', 'cubic']:
    struc_out_n = resample.vertex_da_to_ds(vertex1_nan, ds=ds, method=method)
    compare_vertex_to_struct(vertex1_nan, struc_out_n, method)

## [5. Fill nan values](#TOC)<a name="5"></a>

### Structured grid

In [ ]:
for method in ['nearest', 'linear']:
    struc2d_nan_filled = resample.fillnan_dataarray_structured_grid(struc2d_nan, method=method)
    compare_structured_data_arrays(struc2d_nan, struc2d_nan_filled, method)

## vertex grid

In [ ]:
def compare_vertex_arrays(vertex1, vertex2, method):
    fig, axes = plt.subplots(ncols=2, figsize=(12,6))
    nlmod.plot.plot_vertex_array(vertex1, vertices, ax=axes[0], gridkwargs={'edgecolor': 'k'}, vmin=0, vmax=9)
    axes[0].set_title('original')
    nlmod.plot.plot_vertex_array(vertex2, vertices, ax=axes[1], gridkwargs={'edgecolor': 'k'}, vmin=0, vmax=9)
    axes[1].set_title(f'resampled, method {method}')
    axes[1].set_aspect('equal')

In [ ]:
for method in ['nearest', 'linear']:
    vertex1_nan_filled = resample.fillnan_dataarray_vertex_grid(vertex1_nan, x=xvc, y=yvc, method=method)
    compare_vertex_arrays(vertex1_nan, vertex1_nan_filled, method)

## [6. Vector to grid](#TOC)<a name="6"></a>

Vector data can be points, lines or polygons often saved as shapefiles and visualised using GIS software. A common operation is to project vector data on a modelgrid. For example to add a surface water line to a grid. Here are some functions in `nlmod` to project vector data on a modelgrid.

Vector data:
    - point
    - line
    - polygon

Vector data -> vector data per cell -> aggregeren per cell -> in het grid zetten

In [ ]:
point_geom = [Point(x,y) for x, y in zip([1000, 1200, 1225, 1300],[20200, 20175, 20175, 20425])]
point_gdf = gpd.GeoDataFrame({'values':[1,52,66,24]}, geometry=point_geom)
line_geom = [LineString([point_geom[0], point_geom[1]]),
             LineString([point_geom[2], point_geom[3]]),
             LineString([point_geom[0], point_geom[3]])]
line_gdf = gpd.GeoDataFrame({'values':[1,52,66]}, geometry=line_geom)
pol_geom = [shp_polygon([point_geom[0], point_geom[1], point_geom[2], point_geom[3], point_geom[0]]),
            shp_polygon([point_geom[0], point_geom[1], point_geom[2], Point(1200,20300), point_geom[0]])]
pol_gdf = gpd.GeoDataFrame({'values':[166, 5]}, geometry=pol_geom)


In [ ]:
fig, ax = plt.subplots()
struc2d.plot(ax=ax, lw=0.1, edgecolor='k', alpha=0.5)
point_gdf.plot(ax=ax, color='green')
line_gdf.plot(ax=ax, color='purple')
pol_gdf.plot(ax=ax, alpha=0.6)

ax.set_xlim(ax.get_xlim()[0], 1400)
ax.set_ylim(ax.get_ylim()[0], 20500)

In [ ]:
sim = flopy.mf6.MFSimulation()
gwf = flopy.mf6.MFModel(sim)
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=1, nrow=3, ncol=3, delr=100, delc=100, xorigin=950,yorigin=20050)

### Points

#### Aggregation methods

In [ ]:
fig, axes = plt.subplots(ncols=4, figsize=(20,5))

da1 = nlmod.mdims.gdf_to_data_array_struc(point_gdf, gwf, field='values', agg_method='max')
da2 = nlmod.mdims.gdf_to_data_array_struc(point_gdf, gwf, field='values', agg_method='mean')
da3 = nlmod.mdims.gdf_to_data_array_struc(point_gdf, gwf, field='values', agg_method='nearest')

vmin = min(da1.min(), da2.min(), da3.min())
vmax = max(da1.max(), da2.max(), da3.max())

da1.plot(ax=axes[0], vmin=vmin, vmax=vmax)
axes[0].set_title('aggregation max')
axes[0].axis('scaled')


da2.plot(ax=axes[1], vmin=vmin, vmax=vmax)
axes[1].set_title('aggregation mean')
axes[1].axis('scaled')

da3.plot(ax=axes[2], vmin=vmin, vmax=vmax)
axes[2].set_title('aggregation nearest')
axes[2].axis('scaled')

point_gdf.plot('values', ax=axes[3], vmin=vmin, vmax=vmax, legend=True)
gwf.modelgrid.plot(ax=axes[3])
axes[3].set_title('points')
axes[3].axis('scaled')

#### Interpolation methods

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(15,5))

da1 = nlmod.mdims.gdf_to_data_array_struc(point_gdf, gwf, field='values', interp_method='nearest')
da2 = nlmod.mdims.gdf_to_data_array_struc(point_gdf, gwf, field='values', interp_method='linear')

vmin = min(da1.min(), da2.min())
vmax = max(da1.max(), da2.max())

da1.plot(ax=axes[0], vmin=vmin, vmax=vmax)
axes[0].set_title('interpolation nearest')
axes[0].axis('scaled')

da2.plot(ax=axes[1], vmin=vmin, vmax=vmax)
axes[1].set_title('interpolation linear')
axes[1].axis('scaled')


point_gdf.plot('values', ax=axes[2], vmin=vmin, vmax=vmax, legend=True)
gwf.modelgrid.plot(ax=axes[2])
axes[2].set_title('points')
axes[2].axis('scaled')

### Lines

In [ ]:
fig, axes = plt.subplots(ncols=4, figsize=(20,5))

da1 = nlmod.mdims.gdf_to_data_array_struc(line_gdf, gwf, field='values', agg_method='max_length')
da2 = nlmod.mdims.gdf_to_data_array_struc(line_gdf, gwf, field='values', agg_method='length_weighted')
da3 = nlmod.mdims.gdf_to_data_array_struc(line_gdf, gwf, field='values', agg_method='nearest')

vmin = min(da1.min(), da2.min(), da3.min())
vmax = max(da1.max(), da2.max(), da3.max())

da1.plot(ax=axes[0], vmin=vmin, vmax=vmax)
axes[0].set_title('aggregation max_length')
axes[0].axis('scaled')

da2.plot(ax=axes[1], vmin=vmin, vmax=vmax)
axes[1].set_title('aggregation length_weighted')
axes[1].axis('scaled')

da3.plot(ax=axes[2], vmin=vmin, vmax=vmax)
axes[2].set_title('aggregation nearest')
axes[2].axis('scaled')

line_gdf.plot('values', ax=axes[3], vmin=vmin, vmax=vmax, legend=True)
gwf.modelgrid.plot(ax=axes[3])
axes[3].set_title('lines')
axes[3].axis('scaled')

### Polygons

In [ ]:
fig, axes = plt.subplots(ncols=4, figsize=(20,5))

da1 = nlmod.mdims.gdf_to_data_array_struc(pol_gdf, gwf, field='values', agg_method='max_area')
da2 = nlmod.mdims.gdf_to_data_array_struc(pol_gdf, gwf, field='values', agg_method='area_weighted')
da3 = nlmod.mdims.gdf_to_data_array_struc(pol_gdf, gwf, field='values', agg_method='nearest')

vmin = min(da1.min(), da2.min(), da3.min())
vmax = max(da1.max(), da2.max(), da3.max())

da1.plot(ax=axes[0], vmin=vmin, vmax=vmax)
axes[0].set_title('aggregation max_area')
axes[0].axis('scaled')

da2.plot(ax=axes[1], vmin=vmin, vmax=vmax)
axes[1].set_title('aggregation area_weighted')
axes[1].axis('scaled')

da3.plot(ax=axes[2], vmin=vmin, vmax=vmax)
axes[2].set_title('aggregation nearest')
axes[2].axis('scaled')

pol_gdf.plot('values', ax=axes[3], vmin=vmin, vmax=vmax, legend=True)
gwf.modelgrid.plot(ax=axes[3])
axes[3].set_title('polygons')
axes[3].axis('scaled');

### Intersect vector data with grid

In [ ]:
gdf_point_grid = nlmod.mdims.gdf_to_grid(point_gdf, gwf)
gdf_line_grid = nlmod.mdims.gdf_to_grid(line_gdf, gwf)
gdf_pol_grid = nlmod.mdims.gdf_to_grid(pol_gdf, gwf)

In [ ]:
fig, ax = plt.subplots()

gdf_point_grid.plot(ax=ax, color='green')
gdf_line_grid['ind'] = range(gdf_line_grid.shape[0])
gdf_line_grid.plot('ind', ax=ax, cmap='jet')
gdf_pol_grid['ind'] = range(gdf_pol_grid.shape[0])
gdf_pol_grid.plot('ind',ax=ax, alpha=0.6)

gwf.modelgrid.plot(ax=ax)
ax.set_xlim(ax.get_xlim()[0], 1300)
ax.set_ylim(ax.get_ylim()[0], 20400)

### Aggregate parameters per model cell

Aggregatie options:
- point: max, min, mean
- line: max, min, length_weighted, max_length
- polygon: max, min, area_weighted, area_max


In [ ]:
# point
display(gdf_point_grid)
nlmod.mdims.aggregate_vector_per_cell(gdf_point_grid,{'values':'max'})

In [ ]:
# line
display(gdf_line_grid)
nlmod.mdims.aggregate_vector_per_cell(gdf_line_grid,{'values':'length_weighted'})

In [ ]:
# polygon
display(gdf_pol_grid)
nlmod.mdims.aggregate_vector_per_cell(gdf_pol_grid,{'values':'area_weighted'})

## [7. Grid to reclist](#TOC)<a name="7"></a>
For some modflow packages (drn, riv, ghb, wel) you need to specify stress_period_data to create them using flopy. This stress_period_data consists of reclists (also called lrcd for a structured grid) for every time step. 

The function `da_to_reclist` can be used to convert grid data (both structured and vertex) to a reclist. This function has many arguments:
- `mask`, boolean DataArray to determine which cells should be added to the reclist. Can be 2d or 3d.
- `layer`, if `mask` is a 2d array the value of `layer` is used in the reclist. If `mask` is 3d or `first_active_layer` is True the `layer` argument is ignored.
- `only_active_cells`, if True only add cells with an idomain of 1 to the reclist
- `first_active_layer`, if True use the first active layer, obtained from the idomain, as the layer for each cell.
- `col1`,`col2` and `col3`, The column data of the reclist.

The examples below show the result of each argument.

In [ ]:
# add layer dimension
if 'layer' not in ds.dims:
    ds = ds.expand_dims({'layer':range(3)})

# create some data arrays
ds['da1'] = ('layer', 'y','x'), np.random.randint(0,10,(ds.dims['layer'], ds.dims['y'],ds.dims['x']))
ds['da2'] = ('y','x'), np.random.randint(0,10,(ds.dims['y'],ds.dims['x']))
ds['da3'] = ('y','x'), np.random.randint(0,10,(ds.dims['y'],ds.dims['x']))

# add a nodata value
ds.attrs['nodata'] = -999

# create an idomain of ones except for the first cell which is zero
idomain = np.ones((ds.dims['layer'], ds.dims['y'],ds.dims['x']))
idomain[0,0,0] = 0
ds['idomain'] = ('layer','y','x'), idomain

#### Mask and layer
If `mask` is a 2d array, the `layer` argument specifies the layer that is used in the reclist.

In [ ]:
# structured 2d grid to reclist
mask2d = ds['da2'] == ds['da2'][0,0]
reclist1 = nlmod.mdims.da_to_reclist(ds, mask2d, col1=ds['da1'][0], col2='da2', layer=0, only_active_cells=False)
reclist1

If the `mask` is three dimensional the `layer` argument is ignored.

In [ ]:
# create a 3dmask 
mask3d = ds['da1'] == ds['da1'].values[0,0,0]

# use this mask to create the reclist
reclist2 = nlmod.mdims.da_to_reclist(ds, mask3d, col1='da1',col2=100, layer=0, only_active_cells=False)
reclist2

#### Only active cells
With `only_active_cells=True` we make sure only active cells end up in the reclist. Which cells are active is based on the `idomain` in the model dataset.

In [ ]:
# Only return the cells with an active idomain 
reclist3 = nlmod.mdims.da_to_reclist(ds, mask3d, col1='da1',col2=100, only_active_cells=True)
reclist3

In [ ]:
# also possible for a 2d grid
mask2d = ds['da2'] == ds['da2'][0,0]
reclist1 = nlmod.mdims.da_to_reclist(ds, mask2d, col1=ds['da1'][0], col2='da2', layer=0, only_active_cells=True)
reclist1

#### First active_layer
Use `first_active_layer=True` to add the first active layer to the reclist. The first active layer is obtained from the idomain.

In [ ]:
# create a reclist with col1 (str), col2 (DataArray), col3 (int)
reclist4 = nlmod.mdims.da_to_reclist(ds, mask2d, col1='da2',col2='da3', first_active_layer=True)
reclist4

#### Reclist columns
The `col1`, `col2` and `col3` arguments specify what data should be listed in the reclist. The types can be `str`,`xarray.DataArray`,`None` or other. If the value is a `str` the corresponding DataArray from the Dataset is used to get data for the reclist. If the value is an `xarray.DataArray` the DataArray is used. If the value is `None` the column is not added to the reclist and if the value is from another type the value is used for every record in the reclist.

Be aware that if `mask` is a 3d array, the DataArrays of the column should also be 3d.

In [ ]:
# create a reclist with col1 (str), col2 (DataArray), col3 (int)
reclist5 = nlmod.mdims.da_to_reclist(ds, mask3d, col1=ds['idomain'],col2='da1',col3=9, layer=0, only_active_cells=False)
reclist5

#### Vertex model to reclist

In [ ]:
# add some random DataArray to the vertex dataset
da_vert = np.random.randint(0,10,(dsv['area'].shape))
dsv['da_vert'] = ('icell2d'), da_vert

# create rec list from a vertex dataset
mask_vert = dsv['da_vert'] == dsv['da_vert'][0]
reclist6 = nlmod.mdims.da_to_reclist(dsv, mask_vert, col1='da_vert',col2=2330, only_active_cells=False)
reclist6

## [8. Real world example](#TOC)<a name="8"></a>
In this example we will resample the values of the dutch Digital Terrain Model (DTM) from AHN4 to a structured grid and a vertex grid, using several methods. First we will download the AHN-information.

In [ ]:
extent = [133000, 134000, 402000, 403000]
ahn = nlmod.read.ahn.get_ahn4(extent)

### Transform ahn data to structured grid
We crate a dummy dataset with a structured grid, to which we will resample the AHN-data

In [ ]:
# create an empty model dataset
ds_ahn = nlmod.mdims.get_ds(extent, delr=100.0, layer=1)

In [ ]:
norm = Normalize(ahn.min(), ahn.max())
for method in ['nearest', 'linear', 'average', 'min', 'max']:
    ahn_res = nlmod.resample.structured_da_to_ds(ahn, ds_ahn, method=method)
    
    fig, axes = nlmod.plot.get_map(extent, ncols=2, figsize=(12,6))
    pc = nlmod.plot.data_array(ahn, ax=axes[0], norm=norm)
    nlmod.plot.colorbar_inside(pc, ax=axes[0])
    axes[0].set_aspect('equal')
    axes[0].set_title('original grid')
    pc = nlmod.plot.data_array(ahn_res, dsv, ax=axes[1], edgecolor='k', norm=norm)
    nlmod.plot.colorbar_inside(pc, ax=axes[1])
    axes[1].set_aspect('equal')
    axes[1].set_title(f'resampled grid, method {method}')

### Transform ahn data to vertex grid
We create a vertex grid by refining the cells along a line from the southwest to the northeast.

In [ ]:
gdf = gpd.GeoDataFrame(geometry=[LineString([(extent[0], extent[2]), (extent[1], extent[3])]).buffer(10.)])
dsv = nlmod.mgrid.refine(ds_ahn, model_ws='model7', refinement_features=[(gdf, 1)])

In [ ]:
norm = Normalize(ahn.min(), ahn.max())
for method in ['nearest', 'linear', 'average', 'min', 'max']:
    ahn_res = nlmod.resample.structured_da_to_ds(ahn, dsv, method=method)
    
    fig, axes = nlmod.plot.get_map(extent, ncols=2, figsize=(12,6))
    pc = nlmod.plot.data_array(ahn, ax=axes[0], norm=norm)
    nlmod.plot.colorbar_inside(pc, ax=axes[0])
    axes[0].set_aspect('equal')
    axes[0].set_title('original grid')
    pc = nlmod.plot.data_array(ahn_res, dsv, ax=axes[1], edgecolor='k', norm=norm)
    nlmod.plot.colorbar_inside(pc, ax=axes[1])
    axes[1].set_aspect('equal')
    axes[1].set_title(f'resampled grid, method {method}')